In [1]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [4]:
sys.path.append('../../..')

In [5]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

ModuleNotFoundError: No module named 'modules'

In [5]:
from modules.Elastic_Module import *

ModuleNotFoundError: No module named 'modules'

In [9]:
url_ckan='http://datos.gob.cl/api/3/action/package_list'
url_package='http://datos.gob.cl/api/3/action/package_show?id='
url_package_search='http://datos.gob.cl/api/3/action/package_search?'

In [10]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [11]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [19]:
recursos=[]
for p in packs:
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df

NameError: name 'copy' is not defined

In [17]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato'}) #cambiar nombres de columnas

In [18]:

recursos=recursos.drop(['mimetype','cache_url','url','cache_last_updated','hash','url_type','webstore_url','webstore_last_updated',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

KeyError: "['mimetype' 'cache_url' 'url' 'cache_last_updated' 'hash' 'url_type'\n 'webstore_url' 'webstore_last_updated' 'mimetype_inner' 'position'\n 'resource_type'] not found in axis"

In [13]:
data=recursos.to_dict(orient='records')

In [8]:
bulk_data=[]
for idx in range(0,len(data)):
    bulk_data.append({"index":{ "_index" : "ckan", "_type" : "dataset", "_id" : str(idx) }})
    bulk_data.append(data[idx])

In [9]:
#es = Elasticsearch(hosts = [ES_HOST],http_auth=('elastic', '                    # replace with the empty string'))

In [1]:
ES_HOST = "https://:9200"
INDEX_NAME = 'ckan'

In [2]:
from ssl import create_default_context

context = create_default_context(cafile="/certs/ca/ca.crt")
es = Elasticsearch(
    [ES_HOST],
    http_auth=('elastic',  ),
    scheme="https",
    port=443,
    ssl_context=context,
)

NameError: name 'Elasticsearch' is not defined

In [19]:
es.ping()

True

In [20]:
if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
# since we are running locally, use one shard and no replicas
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [21]:
# bulk index the data
print("bulk indexing...")
res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)

bulk indexing...


In [24]:
# sanity check
res = es.search(index = INDEX_NAME, size=2, body={"query": {"match_all": {}}})
print(" response: '%s'" % (res))

 response: '{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 9726, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'ckan', '_type': 'dataset', '_id': '0', '_score': 1.0, '_source': {'fecha': '2019-05-23T14:53:57.244939', 'formato': 'XLSX', 'inst.': 'Servicio Nacional de Aduanas', 'dia': '2019-05-23', 'year': 2019.0, 'mes': 5.0}}, {'_index': 'ckan', '_type': 'dataset', '_id': '1', '_score': 1.0, '_source': {'fecha': '2019-03-01T18:54:05.594707', 'formato': 'RAR', 'inst.': 'Servicio Nacional de Aduanas', 'dia': '2019-03-01', 'year': 2019.0, 'mes': 3.0}}]}}'


In [23]:
r = requests.get('https://es01:9200',auth=HTTPBasicAuth('elastic', '                    # replace with the empty string'),verify='/certs/ca/ca.crt')

NameError: name 'HTTPBasicAuth' is not defined

In [39]:
r.json()

{'name': 'es01',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'guh4RYBHSKKBTWpuee70vw',
 'version': {'number': '7.2.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '508c38a',
  'build_date': '2019-06-20T15:54:18.811730Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}